# Rename cubes in cubelist with same `standard_name` such that they can easily be merged into a `xr.Dataset`
This is done by appending the vertical coord names then the cell methods

In [1]:
import iris
import os
import xarray as xr
import numpy as np

import crd_utils as crd

In [2]:
# daily data
filepath_d = '/data/cssp-china/mini-dataset-24-01-19/20CR/daily'
files_d = sorted(os.listdir(filepath_d))
cubelist_d = crd.file_to_cubelist(files_d[1], filepath_d)
print(f'Number of cubes: {len(cubelist_d)}')

Cubelist loaded from apepda.pa511f0.pp
Number of cubes: 24


In [34]:
cubenames = [cube.name() for cube in cubelist_d]
cubenames

['m01s05i217',
 'air_pressure_at_sea_level',
 'air_temperature',
 'air_temperature',
 'air_temperature',
 'air_temperature',
 'cloud_area_fraction',
 'geopotential_height',
 'precipitation_flux',
 'relative_humidity',
 'relative_humidity',
 'relative_humidity',
 'relative_humidity',
 'specific_humidity',
 'surface_air_pressure',
 'surface_downwelling_longwave_flux_in_air',
 'surface_downwelling_shortwave_flux_in_air',
 'surface_temperature',
 'surface_temperature',
 'surface_temperature',
 'x_wind',
 'x_wind',
 'y_wind',
 'y_wind']

In [36]:
for cube in cubelist_d:
    if cubenames.count(cube.name()) > 1:
        print(cube.name())

air_temperature
air_temperature
air_temperature
air_temperature
relative_humidity
relative_humidity
relative_humidity
relative_humidity
surface_temperature
surface_temperature
surface_temperature
x_wind
x_wind
y_wind
y_wind


In [114]:
def get_new_cubename(cube):
    suffixes = [cube.name()]
    coord_names = [coord.name() for coord in cube.coords()]
    
    if 'pressure' in coord_names:
        suffixes.append('at_pressure')
    
    if 'height' in coord_names:
        heights = cube.coord('height')
        if len(heights.points) > 1:
            suffixes.append('at_height')
        else:
            height = str(int(heights.points[0].round()))
            units = str(heights.units)
            suffixes.append(f'at_{height}{units}')
    
    for cell_method in cube.cell_methods:
        method = cell_method.method.replace('imum', '')
        suffixes.append(method)
    
    return '_'.join(suffixes)

In [115]:
for cube in cubelist_d:
    display(get_new_cubename(cube))

'm01s05i217_max'

'air_pressure_at_sea_level_mean'

'air_temperature_at_2m_mean'

'air_temperature_at_2m_max'

'air_temperature_at_2m_min'

...

'surface_temperature_min'

'x_wind_at_10m_mean'

'x_wind_at_pressure_mean'

'y_wind_at_10m_mean'

'y_wind_at_pressure_mean'

In [122]:
for cube in cubelist_d:
    if cube.standard_name == None or cubenames.count(cube.name()) > 1:
        new_name = get_new_cubename(cube)
        cube.var_name = new_name
        display(new_name)

'm01s05i217_max'

'air_temperature_at_2m_mean'

'air_temperature_at_2m_max'

'air_temperature_at_2m_min'

'air_temperature_at_pressure_mean'

...

'surface_temperature_min'

'x_wind_at_10m_mean'

'x_wind_at_pressure_mean'

'y_wind_at_10m_mean'

'y_wind_at_pressure_mean'

In [123]:
for cube in cubelist_d:
    print(cube.name())
    print(f'    {cube.standard_name}')
    print(f'    {cube.var_name}')
    print(f'    {cube.long_name}')

m01s05i217_max
    None
    m01s05i217_max
    None
air_pressure_at_sea_level
    air_pressure_at_sea_level
    None
    None
air_temperature
    air_temperature
    air_temperature_at_2m_mean
    None
air_temperature
    air_temperature
    air_temperature_at_2m_max
    None
air_temperature
    air_temperature
    air_temperature_at_2m_min
    None
air_temperature
    air_temperature
    air_temperature_at_pressure_mean
    None
cloud_area_fraction
    cloud_area_fraction
    None
    None
geopotential_height
    geopotential_height
    None
    None
precipitation_flux
    precipitation_flux
    None
    None
relative_humidity
    relative_humidity
    relative_humidity_at_2m_mean
    None
relative_humidity
    relative_humidity
    relative_humidity_at_2m_max
    None
relative_humidity
    relative_humidity
    relative_humidity_at_2m_min
    None
relative_humidity
    relative_humidity
    relative_humidity_at_pressure_mean
    None
specific_humidity
    specific_humidity
    None
 

### Let's try with 6 hourly data (to be sure it's not a fluke)

In [103]:
# 6 hourly data
filepath_6h = '/data/cssp-china/mini-dataset-24-01-19/20CR/6hrly'
files_6h = sorted(os.listdir(filepath_6h))
cubelist_6h = crd.file_to_cubelist(files_6h[1], filepath_6h)
print(f'Number of cubes: {len(cubelist_6h)}')

Cubelist loaded from apepda.pb511f0.pp
Number of cubes: 12


In [127]:
for cube in cubelist_6h:
    display(get_new_cubename(cube))

'm01s05i217_max'

'm01s05i217_mean'

'air_temperature_at_pressure'

'geopotential_height_at_pressure'

'lagrangian_tendency_of_air_pressure_at_pressure'

...

'surface_temperature'

'x_wind_at_10m'

'x_wind_at_pressure'

'y_wind_at_10m'

'y_wind_at_pressure'

In [130]:
cubenames_6h = [cube.name() for cube in cubelist_6h]

for cube in cubelist_6h:
    if cube.standard_name == None or cubenames_6h.count(cube.name()) > 1:
        new_name = get_new_cubename(cube)
        cube.var_name = new_name
        display(new_name)

'm01s05i217_max'

'm01s05i217_mean'

'x_wind_at_10m'

'x_wind_at_pressure'

'y_wind_at_10m'

'y_wind_at_pressure'

## Now we have renamed our cubes, let's see how Xarray behaves

In [119]:
def cubelist_to_dalist(cubelist):
    dalist = []
    for cube in cubelist:
        dalist.append(xr.DataArray.from_iris(cube))
    return dalist

In [129]:
dalist = cubelist_to_dalist(cubelist_d)
for da in dalist:
    print(da.name)

m01s05i217_max
air_pressure_at_sea_level
air_temperature_at_2m_mean
air_temperature_at_2m_max
air_temperature_at_2m_min
air_temperature_at_pressure_mean
cloud_area_fraction
geopotential_height
precipitation_flux
relative_humidity_at_2m_mean
relative_humidity_at_2m_max
relative_humidity_at_2m_min
relative_humidity_at_pressure_mean
specific_humidity
surface_air_pressure
surface_downwelling_longwave_flux_in_air
surface_downwelling_shortwave_flux_in_air
surface_temperature_mean
surface_temperature_max
surface_temperature_min
x_wind_at_10m_mean
x_wind_at_pressure_mean
y_wind_at_10m_mean
y_wind_at_pressure_mean


In [131]:
for da in cubelist_to_dalist(cubelist_6h):
    print(da.name)

m01s05i217_max
m01s05i217_mean
air_temperature
geopotential_height
lagrangian_tendency_of_air_pressure
relative_humidity
specific_humidity
surface_temperature
x_wind_at_10m
x_wind_at_pressure
y_wind_at_10m
y_wind_at_pressure
